## Requirements

코드 실행에 필요한 라이브러리를 사전에 설치합니다.

In [1]:
!pip install transformers
!pip install peft
!pip install datasets
!pip install fire
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=1169

## 라이브러리 불러오기

코드 실행에 필요한 라이브러리를 불러옵니다.

In [2]:
import os
import sys
import textwrap
from typing import List

import fire
import torch
import transformers
from datasets import load_dataset
from peft import (LoraConfig, PeftType, PromptTuningConfig, PromptTuningInit,
                  TaskType, get_peft_config, get_peft_model,
                  get_peft_model_state_dict, prepare_model_for_int8_training)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          LlamaForCausalLM, LlamaTokenizer,
                          default_data_collator,
                          get_linear_schedule_with_warmup)


## 하이퍼파라미터 정의

모델 훈련에 사용할 하이퍼파라미터를 정의합니다.

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 64
learning_rate = 3e-2
num_epochs = 5  # 50
batch_size = 8

## 데이터 불러오기

구글 드라이브를 마운트하여 모델 학습, 검증 및 추론에 사용할 데이터를 불러옵니다.

In [5]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
import pandas as pd

data_dir = "/content/drive/MyDrive/Colab Notebooks/Data"
raw_train_data = pd.read_csv(os.path.join(data_dir, "train_data_frac.csv"))
raw_val_data = pd.read_csv(os.path.join(data_dir, "val_data_frac.csv"))
raw_test_data = pd.read_csv(os.path.join(data_dir, "test_data_frac.csv"))

In [7]:
raw_train_data.head(5)

,question,context,answer,other_answers
0,아르헨티나 헌법 전문에 의해 아르헨티나 국민 대표들은 기존 협약을 이행함에 있어 무...,전 문 우리는 아르헨티나 국민의 대표로서 각 주의 뜻과 선거에 의해 연방제헌의회에 ...,"국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복...",NaN
1,벤 애플렉이 맷 데이먼과 공동으로 설립한 제작사는?,"2000년 데이먼은 애플렉, 크리스 무어, 숀 베일리와 함께 영화 제작사 라이브플래...",라이브플래닛,NaN
2,중앙대학교 연극영화학과와 중앙대학교 예술대학원을 나온 박상아와 남자 프로레슬링선수이...,박상아. 생애. 본관은 밀양(密陽)이며 신장은 166cm이고 체중은 47kg인 그녀...,no,NaN
3,네마냐 비디치는 어느 나라의 축구 선수 였는가?,- 네마냐 비디치: 세르비아의 전 축구 선수,세르비아,NaN
4,국회 프락치 사건은 어떤 의원이 체포된 사건인가?,"국회 프락치 사건은 1949년 6월, 이른바 '남로당 프락치(공작원)'로 제헌국회에...",김약수 의원,NaN


In [8]:
train_data, val_data, test_data = [], [], []
for i in range(len(raw_train_data)):
    data = raw_train_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    train_data.append(tmp)

for i in range(len(raw_val_data)):
    data = raw_val_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    val_data.append(tmp)

for i in range(len(raw_test_data)):
    data = raw_test_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    test_data.append(tmp)

In [9]:
from pprint import pprint

print("==========train_data==========")
pprint(train_data[0])
print("\n==========val_data==========")
pprint(val_data[0])
print("\n==========test_data==========")
pprint(test_data[0])

==========train_data==========
{'input': '전 문 우리는 아르헨티나 국민의 대표로서 각 주의 뜻과 선거에 의해 연방제헌의회에 모여 기존 협약을 이행함에 있어, '
          '국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 '
          '우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해, 모든 이성과 정의의 원천인 신의 가호를 '
          '기원하면서 신의 뜻에 따라 아르헨티나를 위한 이 헌법을 정하고, 명하고, 제정한다.',
 'instruction': '아르헨티나 헌법 전문에 의해 아르헨티나 국민 대표들은 기존 협약을 이행함에 있어 무엇을 위해 동법을 '
                '제정하였는가?',
 'output': '국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 '
           '우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해'}

==========val_data==========
{'input': '제96조 (경쟁정책) ① 연방은 사회적·경제적으로 손해를 끼치는 기업연합 및 기타 형태의 경쟁 제한적 행위를 방지하기 '
          '위한 법률을 제정한다. ② 연방은 다음의 조치를 강구한다. a. 시장지배력이 있는 기업이나 사법·공법상 조직에 의하여 '
          '불공정한 가격이 형성되는 것을 방지한다. b. 불공정경쟁을 근절한다.',
 'instruction': '스위스의 연방은 어떤 경쟁을 근절하는 조치를 강구하는가?',
 'output': '불공정경쟁'}

==========test_data==========
{'input': '이순재. 생애. 1988년 민주정의당 소속으로 국회의원 선거에 출마했지만 낙선하고, 1992년 민주자유당 소속으로 14대 '
         

## 모델 정의

LLaMA 모델을 정의합니다.

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(data_point):
    return f"""아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: {data_point["instruction"]} ### 문맥: {data_point["input"]} ### 응답: {data_point["output"]}"""


def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 1024
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [ ]:
generate_prompt(train_data[0])

In [ ]:
import json

with open("train_data.json", "w") as f:
    json.dump(train_data, f)


In [ ]:
train_data = load_dataset("json", data_files="train_data.json")
train_data

In [ ]:
train_data = train_data.map(generate_and_tokenize_prompt)
val_data = val_data.map(generate_andtokenize_prompt)